
# Anomaly Detection in Telecommunications Networks

This notebook contains code to preprocess data, classify anomalies, and visualize the results. Below are the requirements and instructions to run the notebook.

## Authors

This work was done by **Oussama Guemmar** and **Hatim Haffou**, supervised by **Mr. Said Kasli** at **Orange Morocco**.

## Principle

The principle of this work is to detect anomalies and degradations in telecommunication networks using statistical methods. The approach involves the following steps:

1. Preprocessing the data: Cleaning and transforming the raw data to make it suitable for analysis.
2. Applying rolling averages: Smoothing the data to identify trends and patterns over time.
3. Statistical analysis: Using various statistical techniques to identify and classify anomalies in the data. This helps in distinguishing between normal and abnormal behavior in network traffic.
4. Visualization: Plotting the data to visually inspect the anomalies and understand their impact on network performance.

The ultimate goal is to improve network reliability and performance by identifying and addressing issues promptly.

## Requirements

Install the required packages using the following command:
```bash
pip install -r requirements.txt
```

## Instructions

1. Ensure you have the required dataframes from the specific database with the necessary columns.
2. Run each cell sequentially to preprocess data, classify anomalies, and visualize the results.
3. Company-specific data has been hidden for confidentiality.

### Code Explanation and Execution

Below are the explanations for each code cell:


## Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
from datetime import timedelta
from geopy.distance import great_circle
import matplotlib.pyplot as plt

## Initialize Spark session and load data
This cell initializes the Spark session

In [ ]:
spark = SparkSession.builder.getOrCreate()

## Data preprocessing function - hidden for confidentiality

In [ ]:
def preprocess_data(df):
    ...
    return df

## Classify anomalies
This function identifies anomalies in traffic data by calculating rolling statistics and applying threshold multipliers

In [ ]:
def classify_anomalies(df, data_multiplier, cs_multiplier, classification_window):
    window_spec = Window.partitionBy('CELL_NAME', 'WEEKDAY', 'HOUR').orderBy('DATETIME').rowsBetween(-classification_window + 1, 0)
    df = df.withColumn('CS_RM', F.avg('TRAFIC_CS').over(window_spec))
    df = df.withColumn('DATA_RM', F.avg('TRAFIC_DATA').over(window_spec))
    df = df.withColumn('CS_STDDEV', F.stddev('TRAFIC_CS').over(window_spec))
    df = df.withColumn('DATA_STDDEV', F.stddev('TRAFIC_DATA').over(window_spec))
    df = df.na.fill({'TRAFIC_CS_ROLLING_STD': 0, 'TRAFIC_DATA_ROLLING_STD': 0})
    df = df.withColumn('CS_CLA',
                       F.when(F.col('TRAFIC_CS') > (F.col('CS_RM') + cs_multiplier * F.col('CS_STDDEV')), 'INCREASE')
                       .when(F.col('TRAFIC_CS') < (F.col('CS_RM') - cs_multiplier * F.col('CS_STDDEV')), 'DEGRADATION')
                       .otherwise('STABLE'))
    df = df.withColumn('DATA_CLA',
                       F.when(F.col('TRAFIC_DATA') > (F.col('DATA_RM') + data_multiplier * F.col('DATA_STDDEV')), 'INCREASE')
                       .when(F.col('TRAFIC_DATA') < (F.col('DATA_RM') - data_multiplier * F.col('DATA_STDDEV')), 'DEGRADATION')
                       .otherwise('STABLE'))
    return df


## Get traffic anomalies
This cell extracts the traffic of anomalies cells

In [ ]:
def get_anomalies_traffic(df, anomaly_window, min_anomalies):
    max_timestamp = df.agg(F.max('DATETIME')).collect()[0][0]
    cutoff_datetime = max_timestamp - timedelta(hours=anomaly_window)
    df_filtered = df.filter(F.col('DATETIME') >= cutoff_datetime)
    classification_mapping = {'STABLE': 0, 'INCREASE': 1, 'DEGRADATION': 1}
    mapping_expr = F.create_map([F.lit(x) for x in sum(classification_mapping.items(), ())])
    df_filtered = df_filtered.withColumn('CS_CLANUM', mapping_expr[F.col('CS_CLA')])
    df_filtered = df_filtered.withColumn('DATA_CLANUM', mapping_expr[F.col('DATA_CLA')])
    window_spec = Window.partitionBy('CELL_NAME').orderBy('DATETIME').rowsBetween(-anomaly_window + 1, 0)
    df_filtered = df_filtered.withColumn('CS_SCOUNT', F.sum('CS_CLANUM').over(window_spec))
    df_filtered = df_filtered.withColumn('DATA_SCOUNT', F.sum('DATA_CLANUM').over(window_spec))
    significant_periods = df_filtered.filter(
        (F.col('CS_SCOUNT') >= min_anomalies) | 
        (F.col('DATA_SCOUNT') >= min_anomalies)
    )
    anomaly_cells = significant_periods.select('CELL_NAME').distinct()
    full_traffic_data = df.join(anomaly_cells, on='CELL_NAME', how='inner')
    return full_traffic_data.toPandas()

## Find facing cells
This function finds neighboring cells that are facing the target cell

In [ ]:
def find_facing_cells(df, target_cell):
    def calculate_bearing(lat1, lon1, lat2, lon2):
        dlon = np.radians(lon2 - lon1)
        lat1, lat2 = np.radians(lat1), np.radians(lat2)
        x = np.sin(dlon) * np.cos(lat2)
        y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(dlon)
        return (np.degrees(np.arctan2(x, y)) + 360) % 360

    def is_within_coverage(bearing, azimuth, coverage=60):
        lower, upper = (azimuth - coverage) % 360, (azimuth + coverage) % 360
        return np.where(lower < upper, (bearing >= lower) & (bearing <= upper), (bearing >= lower) | (bearing <= upper))
    
    if target_cell not in df['CELL_NAME'].values:
        return pd.DataFrame()
    target_row = df[df['CELL_NAME'] == target_cell].iloc[0]
    max_distance_km = target_row['MAX_DISTANCE_BY_PROVINCE']
    target_coords = np.array([target_row['LATITUDE'], target_row['LONGITUDE']])
    target_azimuth = target_row['AZIMUTH']
    cell_coords = df[['LATITUDE', 'LONGITUDE']].to_numpy()
    target_coords = np.tile(target_coords, (len(cell_coords), 1))
    distances = np.array([great_circle(target_coords[i], cell_coords[i]).kilometers for i in range(len(cell_coords))])
    within_distance = distances <= max_distance_km
    bearings_to_cell = calculate_bearing(target_coords[:, 0], target_coords[:, 1], cell_coords[:, 0], cell_coords[:, 1])
    bearings_from_cell = calculate_bearing(cell_coords[:, 0], cell_coords[:, 1], target_coords[:, 0], target_coords[:, 1])
    target_within_coverage = is_within_coverage(bearings_to_cell, target_azimuth)
    cell_within_coverage = is_within_coverage(bearings_from_cell, df['AZIMUTH'].to_numpy())
    valid_cells = within_distance & target_within_coverage & cell_within_coverage & (df['CELL_NAME'] != target_cell)
    valid_cells_df = df[valid_cells]
    same_site_same_azimuth = df[(df['SITE_BASE'] == target_row['SITE_BASE']) & (df['AZIMUTH'] == target_row['AZIMUTH']) & (df['CELL_NAME'] != target_cell)]
    valid_cells_df = valid_cells_df[valid_cells_df['SITE_BASE'] != target_row['SITE_BASE']]
    final_cells_df = pd.concat([valid_cells_df, same_site_same_azimuth]).drop_duplicates()
    return final_cells_df

## Get neighbors' traffic data
This function retrieves the traffic of neighboring cells

In [ ]:
def get_neighbors_traffic(classified_df, distinct_cells_pandas_df, geo_df, spark):
    anomaly_neighbors_list = []
    anomaly_geo_list = []

    for _, row in distinct_cells_pandas_df.iterrows():
        target_cell_name = row['CELL_NAME']
        target_rows = geo_df[geo_df['CELL_NAME'] == target_cell_name]
        
        if not target_rows.empty:
            target_row = target_rows.iloc[0]
            target_latitude = target_row['LATITUDE']
            target_longitude = target_row['LONGITUDE']
            target_azimuth = target_row['AZIMUTH']

            facing_cells = find_facing_cells(geo_df, target_cell_name)
            if not facing_cells.empty:
                for _, neighbor_row in facing_cells.iterrows():
                    anomaly_neighbors_list.append({
                        'ANOMALY_CELL': target_cell_name,
                        'NEIGHBOR_CELL': neighbor_row['CELL_NAME']
                    })
                    anomaly_geo_list.append({
                        'CELL_NAME': neighbor_row['CELL_NAME'],
                        'LATITUDE': neighbor_row['LATITUDE'],
                        'LONGITUDE': neighbor_row['LONGITUDE'],
                        'AZIMUTH': neighbor_row['AZIMUTH']
                    })
            anomaly_geo_list.append({
                'CELL_NAME': target_cell_name,
                'LATITUDE': target_latitude,
                'LONGITUDE': target_longitude,
                'AZIMUTH': target_azimuth
            })

    anomaly_neighbors_df = pd.DataFrame(anomaly_neighbors_list)
    anomaly_geo_df = pd.DataFrame(anomaly_geo_list).drop_duplicates()

    anomaly_neighbors_spark_df = spark.createDataFrame(anomaly_neighbors_df)

    neighbor_cells = anomaly_neighbors_spark_df.select("NEIGHBOR_CELL").distinct()
    traffic_neighbors_df = classified_df.join(neighbor_cells, classified_df['CELL_NAME'] == neighbor_cells['NEIGHBOR_CELL'], 'inner') \
                                        .drop(neighbor_cells['NEIGHBOR_CELL'])

    return traffic_neighbors_df.toPandas(), anomaly_neighbors_df, anomaly_geo_df

## Plot traffic data and classifications
This cell plots the traffic data and classifications for the specified date range

In [ ]:
def plot_classifications(df, start_date, end_date, cell_name=None, neighbors_df=None):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    df_filtered = df[(df['DATETIME'] >= start_date) & 
                     (df['DATETIME'] <= end_date)]
    
    if cell_name is not None:
        cells = [cell_name]
        if neighbors_df is not None:
            neighbors = neighbors_df[neighbors_df['ANOMALY_CELL'] == cell_name]['NEIGHBOR_CELL'].unique()
            cells = cells + list(neighbors)
        df_filtered = df_filtered[df_filtered['CELL_NAME'].isin(cells)]
    else:
        cells = df_filtered['CELL_NAME'].unique()
    
    for cell in cells:
        cell_df = df_filtered[df_filtered['CELL_NAME'] == cell].copy()
        cell_df = cell_df.sort_values('DATETIME')
        
        plt.figure(figsize=(14, 6))
        plt.plot(cell_df['DATETIME'], cell_df['TRAFIC_DATA'], label='Data Traffic', linestyle='-', color='blue', alpha=0.5)
        plt.plot(cell_df['DATETIME'], cell_df['TRAFIC_CS'], label='CS Traffic', linestyle='-', color='orange', alpha=0.5)
        
        for status, color in zip(['INCREASE', 'DEGRADATION'], ['red', 'green']):
            data_status_periods = cell_df[cell_df['DATA_CLA'] == status]
            cs_status_periods = cell_df[cell_df['CS_CLA'] == status]

            plt.scatter(data_status_periods['DATETIME'], data_status_periods['TRAFIC_DATA'], c=color, label=f'Data {status}', zorder=5)
            plt.scatter(cs_status_periods['DATETIME'], cs_status_periods['TRAFIC_CS'], c=color, label=f'CS {status}', zorder=5)

        plt.title(f'Traffic Data and Classifications for Cell {cell}')
        plt.xlabel('Datetime')
        plt.ylabel('Traffic')
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

## Execute the complete process and plot results

### Assuming dataframes are loaded from a database and have the required columns
Replace 'database_load_function' with the actual function to load data from your database

In [ ]:
def database_load_function():
    ...
    return traffic_df, geo_df

traffic_df ,geo_df = database_load_function()

processed_df = preprocess_data(traffic_df)
classified_df = classify_anomalies(processed_df,cs_multiplier=1.5,data_multiplier=1.5,classification_window=24)

### Extract anomalies and get neighboring cells' traffic data

In [ ]:
traffic_cells_with_anomalies_pandas_df = get_anomalies_traffic(classified_df, anomaly_window=12, min_anomalies=6)
distinct_cells_pandas_df = traffic_cells_with_anomalies_pandas_df[['CELL_NAME']].drop_duplicates()
traffic_neighbors_pandas_df, anomaly_neighbors_df, anomaly_geo_df = get_neighbors_traffic(classified_df, distinct_cells_pandas_df, geo_df, spark)
full_traffic_data_pandas_df = pd.concat([traffic_cells_with_anomalies_pandas_df, traffic_neighbors_pandas_df]).drop_duplicates()

### Count and list distinct cells with anomalies

In [ ]:
distinct_cells_pandas_df = anomaly_neighbors_df[['ANOMALY_CELL']].drop_duplicates()
distinct_cells_count = distinct_cells_pandas_df['ANOMALY_CELL'].nunique()
print(f"Number of distinct cells with anomalies: {distinct_cells_count}")

distinct_cells_list = distinct_cells_pandas_df['ANOMALY_CELL'].tolist()
distinct_cells_string = ', '.join(map(str, distinct_cells_list))
print(f"Distinct cells with anomalies: {distinct_cells_string}")

### Plot traffic data and classifications

In [ ]:
plot_classifications(df=full_traffic_data_pandas_df, start_date='2024-05-01', end_date='2024-06-01', neighbors_df=anomaly_neighbors_df)